In [ ]:
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
import requests 

Personal list needs to be organized as only directors are currently relevents

In [ ]:
directors = pd.read_csv('data/zippedData/imdb.name.basics.csv.gz', compression = 'gzip')

In [ ]:
directors.head()

In [ ]:
#format profession to list
directors['primary_profession'] = directors['primary_profession'].str.split(',').tolist()
directors = directors.dropna(subset = ['primary_profession'])

In [ ]:
#test for primary profession including director
def is_director(dir_list):
    if 'director' in dir_list:
        return True;
    else:
        return False;

directors['is_director'] = directors['primary_profession'].apply(is_director)

In [ ]:
#remove non directors
directors = directors.loc[directors['is_director']==True]



Should filter out directors we lack suffiecent data about.

In [ ]:
#format known for titles to list
directors['known_for_titles'] = directors['known_for_titles'].str.split(',').tolist()
directors.head()

In [ ]:
#get directors by movie, remove Na
title_crew = pd.read_csv('data/zippedData/imdb.title.crew.csv.gz',compression = 'gzip')
title_crew = title_crew.dropna(subset = ['directors'])

In [ ]:
film_akas = pd.read_csv('data/zippedData/imdb.title.akas.csv.gz',compression = 'gzip')



In [ ]:
#filter name information to only include original titles
film_akas = film_akas.loc[film_akas['is_original_title'] == 1]
film_akas.info()

In [ ]:
#remove data from title_crew not in film AKAS
title_id_unique = set(film_akas['title_id'])
def is_film_id(in_string):
    if in_string in title_id_unique:
        return True;
    else:
        return False


title_crew['is_film'] = title_crew['tconst'].apply(is_film_id)
title_crew = title_crew.loc[title_crew['is_film'] == True]

In [ ]:
#clean NaN values, reformat data
title_crew = title_crew.dropna(subset=['directors'])
title_crew = title_crew.explode('directors')

In [ ]:
#remove directors from directors if not in title_crew
title_crew_unique = set(title_crew['directors'])
def is_movie_director(in_string):
    if in_string in title_crew_unique:
        return True;
    else:
        return False;


directors['is_film_director'] = directors['nconst'].apply(is_movie_director)
directors.head()
directors.loc[directors['is_film_director']==True]



merge formated director informaiton with monetary data

In [ ]:
#get and format moentary information


df = pd.read_csv('data/zippedData/tn.movie_budgets.csv.gz')
df["production_budget"] = df["production_budget"].str.replace(",","").str.replace("$","").astype(int)
df["domestic_gross"] = df["domestic_gross"].str.replace(",","").str.replace("$","").astype(int)
df["worldwide_gross"] = df["worldwide_gross"].str.replace(",","").str.replace("$","").astype(int)
df['profit'] = df['worldwide_gross'] - df['production_budget']
df['return_rate'] = df['profit']/df['production_budget']



In [ ]:
#format title information
title_basics = pd.read_csv('data/zippedData/imdb.title.basics.csv.gz',compression = 'gzip')
title_basics = title_basics.drop(['start_year','runtime_minutes','genres'], axis = 1)
title_basics = title_basics.set_index('primary_title')
title_basics.head()

In [ ]:
#translate movie name into index
def get_tconst(primary_title):
    if primary_title in title_basics.index:
        return title_basics['tconst'][primary_title];
    else:
        return np.NaN;
    


df['tconst'] = df['movie'].apply(get_tconst)

In [ ]:
df = df.dropna(subset=['tconst'])
df.info()



In [ ]:
title_crew = pd.read_csv('data/zippedData/imdb.title.crew.csv.gz',compression = 'gzip')
title_crew = title_crew.dropna(subset = ['directors'])
title_crew = title_crew.set_index('tconst')



In [ ]:
#use movie inde to get director index
valid_tconst = title_crew.index.to_list();
def get_nconst(t_value):
    check = t_value
    if type(t_value) == str:   
        if check in valid_tconst:
            return title_crew['directors'][t_value];
        else:
            return np.NaN;
    else:
        return np.NaN

df['nconst'] = df['tconst'].apply(get_nconst)
df = df.dropna(subset = ['nconst'])

In [ ]:
#reformat director index
df['nconst'] = df['nconst'].str.split(',').tolist()
df.head()
df = df.explode('nconst')
df = df.dropna(subset = ['nconst'])

In [ ]:
df = df.explode('nconst')
df.head(6)

In [ ]:
#fetch known for titles by director
directors = directors.set_index('nconst')
valid_nconst = directors.index.to_list();
def get_known(n_value):
    if type(n_value) == str:   
        if n_value in valid_nconst:
            return directors['known_for_titles'][n_value];
        else:
            return np.NaN;
    else:
        return np.NaN

df['know_for_title'] = df['nconst'].apply(get_known)

df = df.dropna(subset = ['know_for_title'])



retrieve ratings data, based on known movies list

In [ ]:
#get known ratingd for each known for title
def get_ratings_1(in_list):
    outlist = []

    if in_list == 0:
        return np.NaN
    for i in in_list:
        if i in ratings_valid:
            in_out = ratings_valid['averagerating'][i];
            has_movie = True
        else:
            in_out = i;
        outlist.append(in_out)
    return outlist;
df['known_ratings'] = df['know_for_title'].apply(get_ratings_1)

In [ ]:
df.loc[df['known_ratings']==0,'known_ratings']=np.NaN
df = df.dropna(subset = ['known_ratings'])




Since movie ID corresponds that that movies IMDB page, the id can be used to construct a url to web scrape the data

In [ ]:
#for unknown ratings, scrape the data from impb
def scrape_rate(id_list):
    out = []
    try:
        for i in id_list:
            if i[0:2] == 'tt':
                url = f'https://www.imdb.com/title/{i}/'
                response = requests.get(url);
                soup = BeautifulSoup(response.text, 'lxml')
                title = soup.find('span', class_='AggregateRatingButton__RatingScore-sc-1il8omz-1 fhMjqK');
                        #title = soup.find('h1',class_='TitleHeader__TitleText-sc-1wu6n3d-0 gxLYZW');
                out.append(title.text)
                time.sleep(0.5)
            else:
                out.append(i);
        return out
    except:
        return np.NaN
df['known_ratings'] = df['known_ratings'].apply(scrape_rate) 

In [ ]:
df.to_csv('data/zippedData/Final_Data.csv.gz',compression = gzip);